In [1]:
import traitlets
import ipywidgets.widgets as widgets
from ipywidgets import HTML
from IPython.display import display
import threading

from matplotlib import pyplot as plt
import numpy as np

import cv2
import os
from PIL import Image
import time

## スナップショット

####  ディレクトリ作成

In [2]:
import os

dir_path = 'dataset/'
# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(dir_path)
    
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


In [3]:
def xy_uuid(x, y, r):
    return 'xy_%03d_%03d_%03d_%s' % (x * widget_width / 2 + widget_width / 2, y * widget_height / 2 + widget_height / 2,r * widget_height / 4 + widget_height / 4, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value,r_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        free_count.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

In [4]:
camera = Camera()

NameError: name 'Camera' is not defined

In [ ]:
controller = Controller()

In [5]:
#snapshotのGUI
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='snapshot', button_style='success', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(dir_path)))

button_set = widgets.HBox([free_count, free_button])

#操作関連
pos_x_slider = widgets.FloatSlider(description='X', min=-1.0, max=1.0, value=0,step = 0.01)
pos_y_slider = widgets.FloatSlider(description='Y', min=-1.0, max=1.0, value=0,step = 0.01)
pos_r_slider = widgets.FloatSlider(description='R', min=-1.0, max=1.0, value=0,step = 0.01)

stick_input_x_slider = widgets.FloatSlider(description='Stick Input X', min=-1.0, max=1.0, value=0,step=0.01)
stick_input_y_slider = widgets.FloatSlider(description='Stick Input Y', min=-1.0, max=1.0, value=0,step=0.01)
stick_input_r_slider = widgets.FloatSlider(description='Stick Input R', min=-1.0, max=1.0, value=0,step=0.01)

dead_zone_slider = widgets.FloatSlider(description='Dead Zone', min=0.0, max=1.0, value=0.1,step=0.01)
step_slider = widgets.FloatSlider(description='Step Size', min=0.001, max=1.000, value=0.1,step = 0.0001)

option = widgets.VBox([dead_zone_slider, step_slider])


widget_width = 224
widget_height = 224

camera_widget = widgets.Image(format='png', width=widget_width, height=widget_height)
target_widget = widgets.Image(format='png', width=widget_width, height=widget_height)


def display_xy(camera_image):
    #image = np.copy(camera_image)
    
    img = cv2.imread('image/sample.png')
    pos_x = int(pos_x_slider.value * widget_width / 2 + widget_width / 2)
    pos_y = int(pos_y_slider.value * widget_height / 2 + widget_height / 2)
    pos_r = int(pos_r_slider.value * widget_width / 4 + widget_width / 4)
    
    if pos_r <= 2:
        pos_r = 2
    
    img = cv2.circle(image, (pos_x, pos_y), pos_r, (0, 255, 0),2)
    jpeg_image = bgr8_to_jpeg(image)
    
    # 画像をウィジェットに設定
    _, encoded_image = cv2.imencode('.png', img)
    camera_widget.value = encoded_image.tobytes()
    
    return jpeg_image


# X座標を更新する関数
def update_x():
    dead_zone = dead_zone_slider.value
    step = step_slider.value
    x_change = 0
    y_change = 0
    r_change = 0

    if abs(stick_input_x_slider.value) > dead_zone:
        x_change = stick_input_x_slider.value * step
        
    if abs(stick_input_y_slider.value) > dead_zone:
        y_change = stick_input_y_slider.value * step
        
    if abs(stick_input_r_slider.value) > dead_zone:
        r_change = stick_input_r_slider.value * step

    pos_x_slider.value = max(min(pos_x_slider.max, pos_x_slider.value + x_change), pos_x_slider.min)
    pos_y_slider.value = max(min(pos_y_slider.max, pos_y_slider.value + y_change), pos_y_slider.min)
    pos_r_slider.value = max(min(pos_r_slider.max, pos_r_slider.value + r_change), pos_r_slider.min)

    # 定期的にこの関数を再度呼び出す
    threading.Timer(0.05, update_x).start()

    
#controller.buttons[5].observe(save_snapshot, names='value')
free_button.on_click(lambda x: save_snapshot())

time.sleep(1)
# traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
# traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

# 最初の更新を開始
update_x()

option = widgets.VBox([dead_zone_slider, step_slider])
position_x = widgets.VBox([pos_x_slider, stick_input_x_slider])
position_y = widgets.VBox([pos_y_slider, stick_input_y_slider])
position_r = widgets.VBox([pos_r_slider, stick_input_r_slider])

In [6]:
# 空白を作成
def space(width,height):
    width = str(width)+'px'
    height = str(height)+'px'
    return widgets.Box(layout=widgets.Layout(width = width,height = height))


#コントローラのGUI
controller_html=HTML(
    value="コントローラの設定"
)

###データ集め
option_layout = widgets.VBox([space(0,30),button_set,space(0,30),controller_html,option])
#image_layout = widgets.HBox([image_placeholder,image_placeholder])
image_layout = widgets.HBox([camera_widget, target_widget])
dataCollect_layout = widgets.HBox([image_layout,space(30,0),option_layout])


In [7]:
display(dataCollect_layout)


# クラス分類 

### ディレクトリ作成

In [8]:
classes = ['left_near', 'left_just', 'left_far', 
            'middle_near', 'middle_just', 'middle_far', 
            'right_near', 'right_just', 'right_far']

os.makedirs('class',exist_ok = True)
for s in classes:
    os.makedirs('class/'+s,exist_ok=True)

In [9]:
# インタラクティブなウィジェットを作成
x1_slider = widgets.IntSlider(min=0, max=224/2, value=10, description='x1:')
x2_slider = widgets.IntSlider(min=0, max=224/2, value=10, description='x2:')

x_slider = widgets.IntSlider(min=0, max=300, value=150, description='X:')
y_slider = widgets.IntSlider(min=0, max=300, value=150, description='Y:')
rad_start_slider = widgets.IntSlider(min=0, max=300, value=50, description='Rad Start:')
rad_end_slider = widgets.IntSlider(min=0, max=300, value=100, description='Rad End:')

img_widget_1 = widgets.Image(format='png', width=224, height=224)
img_widget_2 = widgets.Image(format='png', width=224, height=224)

#カウンタ
counters = [widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[0]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[1]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[2]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[3]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[4]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[5]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[6]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[7]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir('class/'+classes[8])))]


In [10]:
# boundaryTest関数を修正して、ウィジェットに画像を表示する
def LineTest(x1, x2, img_path, img_widget):
    # 画像を読み込み、リサイズする
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    mid_x = 112
    mid_y = 112
    new_x1 = 2 * mid_x - x1
    new_x2 = 2 * mid_x - x2
    
    # 線の色と太さを定義
    blue_color = (255, 0, 0)
    thickness = 2
    
    # 画像上に線を描画
    cv2.line(img, (int(x1), 0), (int(x2), img.shape[0]), blue_color, thickness)
    cv2.line(img, (new_x1, 0), (new_x2, img.shape[0]), blue_color, thickness)
    
    # 画像をウィジェットに設定
    _, encoded_image = cv2.imencode('.png', img)
    img_widget_1.value = encoded_image.tobytes()
    
def CircleTest(x, y, rad_start, rad_end, img_widget, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))

    # 色と太さの設定
    red_color = (0, 0, 255)
    green_color = (0, 255, 0)
    yellow_color = (0, 255, 255)
    circle_thickness = -1
    thickness = 2

    # 円を描画
    cv2.circle(img, (x, y), rad_start, green_color, thickness)
    cv2.circle(img, (x, y), rad_end, green_color, thickness)
    cv2.circle(img, (x, y), 2, red_color, circle_thickness)

    # 画像をウィジェットに設定
    _, encoded_image = cv2.imencode('.png', img)
    img_widget_2.value = encoded_image.tobytes()

    
def update_line(change):
    LineTest(x1_slider.value, x2_slider.value, img_path, img_widget_1)
    
    # スライダーの値が変更されたときのコールバック関数
def update_image(change):
    CircleTest(x_slider.value, y_slider.value, rad_start_slider.value, rad_end_slider.value, img_widget_2, img_path)


### クラス分類 - 分類処理関数

In [11]:
def get_index(direction,distance):
    
    row = 0
    column = 0
    
    if direction == 'left':
        column = 0
    elif direction == 'middle':
        column = 1
    elif direction == 'right':
        column = 2
        
    if distance == 'near':
        row = 0 
    elif direction == 'just':
        row = 1 
    elif direction == 'far':
        row = 2
        
    return 3*column + row
    
def fx(x1, x2, x, img): # 座標(x1, 0),(x2, img.shape[0])の2点を通る直線の式における、xがx(引数)の時のyを返す
    y = img.shape[0] // (x2 - x1) * (x - x1)
    return y
    
def directionClassify(x1, x2, x, y, img_path): # 方向分類
    img = cv2.imread(img_path)
    img = cv2.resize(img,(300,300)) 
    
    img_width = 300
    
    
    x1 = x1 * img.shape[1] // img_width
    x2 = x2 * img.shape[1] // img_width
    
    mid_x = img.shape[1] // 2
    new_x1 = 2 * mid_x - x1
    new_x2 = 2 * mid_x - x2
    
    if x1 < x2:
        if (x == x1 and y == 0) or (x == new_x1 and y == 0) or (x >= x2 and x <= new_x2):
            direction = 'middle'
        elif x > x1 and x < x2 and y <= fx(x1, x2, x, img):
            direction = 'middle'
        elif x > new_x2 and x < new_x1 and y <= fx(new_x1, new_x2, x, img):
            direction = 'middle'
        elif x >= 0 and x <= x1:
            direction = 'left'
        elif x > x1 and x < x2 and y > fx(x1, x2, x, img):
            direction = 'left'
        elif x >= new_x1 and x <= img.shape[1]:
            direction = 'right'
        elif x > new_x2 and x < new_x1 and y > fx(new_x1, new_x2, x, img):
            direction = 'right'
    elif x1 > x2:
        if (x == x2 and y == img.shape[0]) or (x == new_x2 and y == img.shape[0]) or (x >= x1 and x <= new_x1):
            direction = 'middle'
        elif x > x2 and x < x1 and y >= fx(x1, x2, x, img):
            direction = 'middle'
        elif x > new_x1 and x < new_x2 and y >= fx(new_x1, new_x2, x, img):
            direction = 'middle'
        elif x >= 0 and x <= x2:
            direction = 'left'
        elif x > x2 and x < x1 and y < fx(x1, x2, x, img):
            direction = 'left'
        elif x >= new_x2 and x <= img.shape[1]:
            direction = 'right'
        elif x > new_x1 and x < new_x2 and y < fx(new_x1, new_x2, x, img):
            direction = 'right'
    elif x1 == x2:
        if x >= 0 and x < x1:
            direction = 'left'
        elif x > new_x1 and x <= img.shape[1]:
            direction = 'right'
        else:
            direction = 'middle'
            
    return direction
        
def distanceClassify(rad_start, rad_end, rad, img_path): # 距離分類
    img = cv2.imread(img_path)
    img = cv2.resize(img,(300,300)) 
    img_width = 300
    
    
    #start = img.shape[1] // rad_start // 2 # justの下限
    #end = img.shape[1] // rad_end // 2 # justの上限
    
    start = rad_start * img.shape[1] // img_width
    end = rad_end * img.shape[1] // img_width
    
    if rad > 0 and rad < start:
        distance = 'far'
    elif rad > end:
        distance = 'near'
    else:
        distance = 'just'
        
    return distance
    
def allClassify(x1, x2, x, y, rad_start, rad_end, rad, img_path): # 全体の分類
    
    """ 修正　"""
    
    img = cv2.imread(img_path)
    #original_img = img
    output_folder = ''
    
    direction = directionClassify(x1, x2, x, y, img_path)
    distance = distanceClassify(rad_start, rad_end, rad, img_path)
    
    
    
    global file_name
    output_folder = 'class/'+direction+'_'+distance+'/' + file_name
    original_img = Image.open(img_path)
    original_img.save(output_folder)
    
    n = get_index(direction,distance)
    counters[n].value = len(os.listdir('class/'+direction+'_'+distance))
        
    print(output_folder)
    

def getrate(img_path):
    ###Resizeする倍率を取得 座標や半径の調整に利用###
    img = cv2.imread(img_path)
    
    return img.shape[0] / 300,img.shape[1] / 300
    

def get_x(path):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[1])) ) 

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) ) 

def get_r(path):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[3])) )

In [12]:
# 画像のパスを設定
img_path = 'image/sample.png' # 画像のパスを適切なものに変更してください

# スライダーの値が変更されたときにコールバック関数を実行
x1_slider.observe(update_line, names='value')
x2_slider.observe(update_line, names='value')

# スライダーの変更を監視
x_slider.observe(update_image, names='value')
y_slider.observe(update_image, names='value')
rad_start_slider.observe(update_image, names='value')
rad_end_slider.observe(update_image, names='value')

# 初期表示
LineTest(x1_slider.value, x2_slider.value, img_path, img_widget_1)
# 初期画像を設定
update_image(None)


In [24]:
# ウィジェットを配置
data_html=HTML(
    value="データ集め"
)

# 水平線を引くための HTML ウィジェットを作成
horizontal_line = widgets.HTML(
    value="<hr style='border-top: 1px solid black;'>",
    layout=widgets.Layout(width='100%')
)

class_html=HTML(
    value="クラス分類"
)

###クラス分類

#####境界線
image_layout_2_1 = widgets.HBox([img_widget_1])
xy_option_layout = widgets.VBox([x1_slider,x2_slider]) 
line_layout = widgets.VBox([image_layout_2_1,xy_option_layout])
gui_layout_2_1 = widgets.HBox([line_layout])

#####境界円
image_layout_2_2 = widgets.HBox([img_widget_2])
xy_option_layout_2 = widgets.VBox([x_slider,y_slider,rad_start_slider,rad_end_slider])
circle_layout = widgets.VBox([image_layout_2_2,xy_option_layout_2])
gui_layout_2_2 = widgets.HBox([circle_layout])

In [30]:
classificate_layout = widgets.HBox([gui_layout_2_1,gui_layout_2_2])
classificate_layout = widgets.VBox([horizontal_line,class_html,classificate_layout,space(0,30)])
# GUIを表示
display(classificate_layout)

In [14]:

def create_html(num):
    
    global classes
    return HTML(
        value= classes[num]   
        )

data_button = widgets.Button(description='run', button_style='success', layout=button_layout)
reload_button = widgets.Button(description='load', button_style='success', layout=button_layout)

In [15]:
left_layout = widgets.VBox([create_html(0),counters[0],create_html(1),counters[1],create_html(2),counters[2]])
middle_layout = widgets.VBox([create_html(3),counters[3],create_html(4),counters[4],create_html(5),counters[5]])
right_layout = widgets.VBox([create_html(6),counters[6],create_html(7),counters[7],create_html(8),counters[7]])
button_layout_2 = widgets.VBox([space(0,100),data_button,space(0,30),reload_button])

all_layout = widgets.HBox([button_layout_2,space(30,0),left_layout,space(30,0),middle_layout,space(30,0),right_layout])

In [16]:
file_name = ''

def reload():
    classes = ['left_near', 'left_just', 'left_far', 
            'middle_near', 'middle_just', 'middle_far', 
            'right_near', 'right_just', 'right_far']
    
    for i in range(len(classes)):  
        counters[i].value = len(os.listdir('class/'+classes[i]))
    

def run_classificate(data_path='dataset/'):
    # Get all entries in the 'image/back/' directory
    print("run")
    entries = os.listdir(data_path)

    # Filter to include only .jpg files
    img_files = [file for file in entries if file.endswith('.png')]
    print(img_files)

    global file_name
    for img in img_files:
        
        file_name = img
        img_path = 'dataset/'+file_name
        
        kx,ky = getrate(img_path)
        x= int(get_x(file_name) / kx)
        y = int(get_y(file_name) / ky)
        rad = int(get_r(file_name) / kx)
        
        allClassify(int(x1_slider.value), int(x2_slider.value), x, y, int(rad_start_slider.value), int(rad_end_slider.value), rad, img_path)

data_button.on_click(lambda x: run_classificate())
reload_button.on_click(lambda x: reload())

In [17]:
display(all_layout)

In [31]:
lay = widgets.VBox([dataCollect_layout,space(0,100),classificate_layout,all_layout])
display(lay)